# 코로나19 시ㆍ도별 발생 현황

In [1]:
!pip install pandas

import requests
from bs4 import BeautifulSoup as bs
from urllib import parse
import pandas as pd
import datetime

today = datetime.datetime.now()
weeks = today-datetime.timedelta(days=6)

serviceKey = 'jzKP7EwNPdYnAExeAldDB%2FzFTAu3KjXlmGTs04CgNKjULRXTka08HFWvSU0MYLyW87Wmz5hm%2FxxL9drGB9h5sQ%3D%3D'
params = {'ServiceKey':parse.unquote(serviceKey),
          'startCreateDt': weeks.strftime('%Y%m%d'),
          'endCreateDt': today.strftime('%Y%m%d')}

url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson'

res = requests.get(url, params=params)
soup = bs(res.text, 'lxml')

items = soup.find_all('item')

response = {'createdt':'등록일시',
            'deathcnt':'사망자',
            'defcnt':'확진자 수',
            'gubun':'지역',
            'gubuncn':'지역_중국어',
            'gubunen':'지역_영어',
            'incdec':'전일대비 증감 수',
            'isolclearcnt':'격리 해제 수',
            'isolingcnt':'격리 중 환자 수',
            'localocccnt':'확진자',
            'overflowcnt':'해외유입',
            'qurrate':'10만명당 발생률',
            'seq':'고유번호',
            'stdday':'기준일시',
            'updatedt':'수정일시'}

lst = []
for y in items:
    l = {}
    for x in y:
        l[x.name] = x.text
    lst.append(l)

df = pd.DataFrame(lst)
df.columns = df.columns.map(response)
df

등록일시   사망자   확진자 수  지역 지역_중국어             지역_영어  \
0    2021-06-01 09:43:52.317     4    3848  검역    隔離區         Lazaretto   
1    2021-06-01 09:43:52.317     1    1042  제주     济州              Jeju   
2    2021-06-01 09:43:52.317    19    4707  경남     庆南  Gyeongsangnam-do   
3    2021-06-01 09:43:52.317    85    4701  경북     庆北  Gyeongsangbuk-do   
4    2021-06-01 09:43:52.316    15    1468  전남     全南      Jeollanam-do   
..                       ...   ...     ...  ..    ...               ...   
128  2021-05-26 10:11:28.701    61    6094  인천     仁川           Incheon   
129  2021-05-26 10:11:28.701   221    9792  대구     大邱             Daegu   
130  2021-05-26 10:11:28.701   124    5593  부산     釜山             Busan   
131  2021-05-26 10:11:28.701   481   43001  서울     首尔             Seoul   
132  2021-05-26 10:11:28.701  1940  137681  합계     合计             Total   

    전일대비 증감 수 격리 해제 수 격리 중 환자 수  확진자 해외유입 10만명당 발생률  고유번호               기준일시  \
0           4    3600       244    0    4         -  9868  2021년 06월 01일 00시   
1          12     906       135   12    0    155.35  9867  2021년 06월 01일 00시   
2           8    4489       199    8    0    140.03  9866  2021년 06월 01일 00시   
3           9    4418       198    9    0    176.56  9865  2021년 06월 01일 00시   
4           5    1303       150    5    0     78.73  9864  2021년 06월 01일 00시   
..        ...     ...       ...  ...  ...       ...   ...                ...   
128        19    5835       198   19    0    206.15  9740  2021년 05월 26일 00시   
129        30    9244       327   30    0    401.89  9739  2021년 05월 26일 00시   
130        22    5229       240   21    1    163.93  9738  2021년 05월 26일 00시   
131       262   39680      2840  260    2    441.78  9737  2021년 05월 26일 00시   
132       706  127582      8159  684   22    265.55  9736  2021년 05월 26일 00시   

                       수정일시  
0                      null  
1                      null  
2                      null  
3                      null  
4                      null  
..                      ...  
128                    null  
129                    null  
130                    null  
131                    null  
132  2021-05-31 10:33:11.03  

[133 rows x 15 columns]

In [2]:
jejuDf = df.loc[(df['지역']=='제주')]
jeju = jejuDf.loc[:,['등록일시','확진자','해외유입','사망자','격리 중 환자 수','격리 해제 수']].dropna()
jeju

등록일시 확진자 해외유입 사망자 격리 중 환자 수 격리 해제 수
1    2021-06-01 09:43:52.317  12    0   1       135     906
20   2021-05-31 09:41:22.918   6    1   1       131     898
39    2021-05-30 09:34:54.06   8    0   1       126     896
58    2021-05-29 09:53:56.44  12    1   1       123     891
77   2021-05-28 09:38:49.615   7    0   1       120     881
96   2021-05-27 10:05:19.315  26    0   1       126     868
115  2021-05-26 10:11:28.703   6    0   1       113     855

In [3]:
for col in jeju.columns:
    if col in ['등록일시']: # 날짜 : datetime
        jeju[col] = jeju[col].apply(pd.to_datetime)
    else : # 나머지 columns : 수치형
        jeju[col] = jeju[col].apply(pd.to_numeric)
        
jeju.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 1 to 115
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   등록일시       7 non-null      datetime64[ns]
 1   확진자        7 non-null      int64         
 2   해외유입       7 non-null      int64         
 3   사망자        7 non-null      int64         
 4   격리 중 환자 수  7 non-null      int64         
 5   격리 해제 수    7 non-null      int64         
dtypes: datetime64[ns](1), int64(5)
memory usage: 392.0 bytes


In [4]:
jeju.to_csv('data/corona_Weekly_status.csv', index=False, encoding='utf8')

import os
os.listdir('data')

['Activity_Negative_Text.csv',
 'Activity_Positive_Text.csv',
 'corona_Integration_status.csv',
 'corona_Weekly_status.csv']

In [5]:
!pip install plotly

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

data = pd.read_csv('data/corona_Weekly_status.csv')

data.등록일시 = data.등록일시.apply(pd.to_datetime)
data

등록일시  확진자  해외유입  사망자  격리 중 환자 수  격리 해제 수
0 2021-06-01 09:43:52.317   12     0    1        135      906
1 2021-05-31 09:41:22.918    6     1    1        131      898
2 2021-05-30 09:34:54.060    8     0    1        126      896
3 2021-05-29 09:53:56.440   12     1    1        123      891
4 2021-05-28 09:38:49.615    7     0    1        120      881
5 2021-05-27 10:05:19.315   26     0    1        126      868
6 2021-05-26 10:11:28.703    6     0    1        113      855

In [6]:
df = data.copy()
df = df.sort_values('등록일시')

fig = go.Figure()
fig.add_trace(go.Bar(x=df.등록일시.dt.strftime('%m.%d'),
                     y=df.확진자,
                     text=df.확진자,
                     textposition='outside',
                     hovertemplate='기준일: %{x} <br>확진: %{y}명',
                     name='확진'
                    )
             )
fig.add_trace(go.Bar(x=df.등록일시.dt.strftime('%m.%d'),
                     y=df.사망자,
                     text=df.사망자,
                     textposition='outside',
                     hovertemplate='기준일: %{x} <br>사망: {y}명',
                     name='사망'
                    )
             )
fig.add_trace(go.Bar(x=df.등록일시.dt.strftime('%m.%d'),
                     y=df.해외유입,
                     text=df.해외유입,
                     textposition='outside',
                     hovertemplate='기준일: %{x} <br>해외유입: {y}명',
                     name='해외유입'
                    )
             )
fig.update_layout(xaxis=dict(type='category'),
                  title_text='제주도 코로나19(COVID-19) 추이 - 2021.05월 4주차'
                 )
fig.show()
data

등록일시  확진자  해외유입  사망자  격리 중 환자 수  격리 해제 수
0 2021-06-01 09:43:52.317   12     0    1        135      906
1 2021-05-31 09:41:22.918    6     1    1        131      898
2 2021-05-30 09:34:54.060    8     0    1        126      896
3 2021-05-29 09:53:56.440   12     1    1        123      891
4 2021-05-28 09:38:49.615    7     0    1        120      881
5 2021-05-27 10:05:19.315   26     0    1        126      868
6 2021-05-26 10:11:28.703    6     0    1        113      855